<a href="https://colab.research.google.com/github/lee-messi/machine-learning/blob/main/cola_grammar_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow --quiet
!pip install tensorflow-hub --quiet
!pip install tensorflow-text --quiet
!pip install tensorflow-addons --quiet
!pip install tensorflow-datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 44.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.7 requires jax>=0.4.6, but you have jax 0.3.25 which is incompatible.
flax 0.6.11 requires jax>=0.4.2, but you have jax 0.3.25 which is incompatible.
orbax-checkpoint 0.2.6 requires jax>=0.4.9, but you have jax 0.3.25 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 11.1 MB/s eta 0:00:00


In [10]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow_addons as tfa
import tensorflow_datasets as tfds

tf.get_logger().setLevel('ERROR')
os.environ["TFHUB_MODEL_LOAD_FORMAT"]="UNCOMPRESSED"

In [5]:
if os.environ['COLAB_TPU_ADDR']:
  cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
  tf.config.experimental_connect_to_cluster(cluster_resolver)
  tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
  strategy = tf.distribute.TPUStrategy(cluster_resolver)
  print('Using TPU')
elif tf.config.list_physical_devices('GPU'):
  strategy = tf.distribute.MirroredStrategy()
  print('Using GPU')
else:
  raise ValueError('Running on CPU is not recommended.')

Using TPU


In [18]:
train = pd.read_csv('drive/MyDrive/Colab Notebooks/glue-tasks/in_domain_train.tsv',
                    sep = '\t', header = None)
val = pd.read_csv('drive/MyDrive/Colab Notebooks/glue-tasks/in_domain_dev.tsv',
                  sep = '\t', header = None)
test = pd.read_csv('drive/MyDrive/Colab Notebooks/glue-tasks/out_of_domain_dev.tsv',
                   sep = '\t', header = None)

In [25]:
train.columns = ['misc', 'labels', 'na', 'text']
val.columns = ['misc', 'labels', 'na', 'text']
test.columns = ['misc', 'labels', 'na', 'text']

In [26]:
'{}{}{}'.format(train.shape, val.shape, test.shape)

'(8551, 4)(527, 4)(516, 4)'

In [42]:
def df_to_dataset(dataframe, shuffle = True, batch_size = 256):
    df = dataframe.copy()
    labels = df.labels
    df = df.text
    ds = tf.data.Dataset.from_tensor_slices((df, labels))
    if shuffle == True:
        ds = ds.shuffle(buffer_size = len(df))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return(ds)

In [43]:
train_ds = df_to_dataset(train)
val_ds = df_to_dataset(val)
test_ds = df_to_dataset(test)

In [28]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [38]:
bert_preprocess = hub.KerasLayer(tfhub_handle_preprocess)
bert_encoder = hub.KerasLayer(tfhub_handle_encoder)

In [51]:
text_input = tf.keras.layers.Input(shape = (), dtype = tf.string)
encoder_input = bert_preprocess(text_input)
encoder_output = bert_encoder(encoder_input)

l = tf.keras.layers.Dense(100, activation = 'relu')(encoder_output['pooled_output'])
l = tf.keras.layers.Dense(1, activation = 'sigmoid')(l)
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [52]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.005),
              loss = tf.keras.losses.BinaryCrossentropy(),
              metrics = ['accuracy'])

In [53]:
history = model.fit(train_ds,
                    validation_data = val_ds,
                    epochs = 10)

Epoch 1/10
34/34 [==============================] - 222s 6s/step - loss: 0.8136 - accuracy: 0.6493 - val_loss: 0.5866 - val_accuracy: 0.7040
Epoch 2/10
34/34 [==============================] - 216s 6s/step - loss: 0.5761 - accuracy: 0.7121 - val_loss: 0.5773 - val_accuracy: 0.7059
Epoch 3/10
34/34 [==============================] - 215s 6s/step - loss: 0.5635 - accuracy: 0.7200 - val_loss: 0.5704 - val_accuracy: 0.7059
Epoch 4/10
34/34 [==============================] - 215s 6s/step - loss: 0.5604 - accuracy: 0.7242 - val_loss: 0.5649 - val_accuracy: 0.7116
Epoch 5/10
34/34 [==============================] - 215s 6s/step - loss: 0.5541 - accuracy: 0.7314 - val_loss: 0.5544 - val_accuracy: 0.7306
Epoch 6/10
34/34 [==============================] - 216s 6s/step - loss: 0.5500 - accuracy: 0.7318 - val_loss: 0.5632 - val_accuracy: 0.7192
Epoch 7/10
34/34 [==============================] - 216s 6s/step - loss: 0.5444 - accuracy: 0.7344 - val_loss: 0.5762 - val_accuracy: 0.7116
Epoch 8/10
34

In [ ]:
model.predict([''])